In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from metrics import *
from datasetHandler import *
import numpy as np
from skimage.io import imsave

In [ ]:
dataset_path = 'val_2'
s2_paths, s2_zones = get_images_path(dataset_path, 'sen2')
s1_images, s1_zones = get_images_path(dataset_path, 'sen1')
s2_images, cloud_masks = split_S2_images(s2_paths)
s2_series = get_time_series(s2_images)
s1_series = get_time_series(s1_images)

In [ ]:
print(len(s2_series), len(s1_series))

In [ ]:
def testModels(ground_truth, image_to_check):
    distance = 5
    shift = 3
    #PSNR - MAX: INF
    psnr = shifted_psnr(ground_truth, image_to_check, distance, shift)
    #SSIM - MAX: 1.0
    ssim = shifted_ssim(ground_truth, image_to_check, distance, shift)
    #FID - MAX: 0.0
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(246,246,3))
    fid = shifted_fid(model, ground_truth, image_to_check, distance, shift)
    #CSC-PSNR metric - MAX: 0.0
    csc_psnr = csc_psnr_metric(ground_truth, image_to_check, distance, shift)
    #RMSE - MAX:  0.0
    rmse1 = shifted_rmse(ground_truth, image_to_check, distance, shift)
    #SAM - MAX: 0.0
    sam1 = shifted_sam(ground_truth, image_to_check, distance, shift)
    #UQI - MAX: 1.0
    uqi1 = shifted_uqi(ground_truth, image_to_check, distance, shift)
    #DD - MAX: 0.0
    dd = shifted_DD(ground_truth, image_to_check, distance, shift)
    #CC - MAX: 1.0
    cc = shifted_CC(ground_truth, image_to_check, distance, shift)
    
    return psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc

In [ ]:
def pprint(title, psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc):
    print(title)
    print('     PNSR     -> ', psnr) 
    print('     SSIM     -> ', ssim)
    print('     FID      -> ', fid)
    print('     CSC-PSNR -> ', csc_psnr)
    print('     RMSE     -> ', rmse1)
    print('     SAM      -> ', sam1)
    print('     UQI      -> ', uqi1)
    print('     DD       -> ', dd)
    print('     CC       -> ', cc)

In [ ]:
from tensorflow.keras.models import load_model
lstm = load_model('LSTM_09-06-20_3-layers.h5')
gan = load_model('model_007500.h5')
despeckling = load_model('despeckling.h5')
combined = load_model('combined.h5')

batch_size = 60
iterator = iter(image_generator(s2_series, s1_series, batch_size = batch_size, normalization='minmax'))

In [ ]:
batch_s2, batch_s1 = next(iterator)
batch_s2[i, 3, ...].shape

In [ ]:
batch_s2, batch_s1 = next(iterator)
combiner_input = np.zeros((1, 256, 256, 6))

for i in range(batch_size):    
    lstm_prediction = lstm.predict(batch_s2[i, 0:3, ...].reshape((1, 3, 256, 256, 3)))
    
    s1 = despeckling.predict(batch_s1[i, ...].reshape(4,256,256,1))
    #s1 = batch_s1[i, ...]

    gan_prediction = gan.predict(s1)
    gan_prediction = (np.array(gan_prediction) + 1) / 2.0

    combiner_input[0,...,:3] = lstm_prediction[0,0,...]
    combiner_input[0,...,3:6] = gan_prediction[3,...]
    
    combiner_out = combined.predict(combiner_input)
    

    fig, axes = plt.subplots(nrows = 2, ncols = 6, figsize = (30, 10))
    
    axes[0,0].imshow(batch_s2[i, 0, ...])
    axes[0,0].set_title('S2 - 1', fontsize = 24)
    axes[0,0].axis(False)
    
    axes[0,1].imshow(batch_s2[i, 1, ...])
    axes[0,1].set_title('S2 - 2', fontsize = 24)
    axes[0,1].axis(False)
    
    axes[0,2].imshow(batch_s2[i, 2, ...])
    axes[0,2].set_title('S2 - 3', fontsize = 24)
    axes[0,2].axis(False)
    
    axes[0,3].imshow(batch_s2[i, 3, ...])
    axes[0,3].set_title('S2 - 4 (GT)', fontsize = 24)
    axes[0,3].axis(False)
    
    imsave("tests/test_"+str(i)+"_ground_truth.png", batch_s2[i, 3, ...])
    
    
    axes[0,4].imshow(lstm_prediction[0,0,...])
    axes[0,4].set_title('LSTM', fontsize = 24)
    axes[0,4].axis(False)

    imsave("tests/test_"+str(i)+"_LSTM.png", lstm_prediction[0,0,...])
    
    axes[0,5].imshow(combiner_out[0,...])
    axes[0,5].set_title('Combined', fontsize = 24)
    axes[0,5].axis(False)
    
    imsave("tests/test_"+str(i)+"_combined.png", combiner_out[0,...])

    #----------------------------------------
    axes[1,0].imshow((batch_s1[i, 0, ...]+ 1) / 2.0, cmap = 'gray')
    axes[1,0].set_title('S1 - 1', fontsize = 24)
    axes[1,0].axis(False)
    
    axes[1,1].imshow((batch_s1[i, 1, ...]+ 1) / 2.0, cmap = 'gray')
    axes[1,1].set_title('S1 - 2', fontsize = 24)
    axes[1,1].axis(False)
    
    axes[1,2].imshow((batch_s1[i, 2,...]+ 1) / 2.0, cmap = 'gray')
    axes[1,2].set_title('S1 - 3', fontsize = 24)
    axes[1,2].axis(False)
    
    axes[1,3].imshow((batch_s1[i, 3,...]+ 1) / 2.0, cmap = 'gray')
    axes[1,3].set_title('S1 - 4', fontsize = 24)
    axes[1,3].axis(False)
    
    axes[1,4].imshow(gan_prediction[3,...])
    axes[1,4].set_title('GAN', fontsize = 24)
    axes[1,4].axis(False)
    
    imsave("tests/test_"+str(i)+"_GAN.png", gan_prediction[3,...])
    
    axes[1,5].axis(False)

    plt.show()
    plt.close()
    
    
    #psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], lstm_prediction[0,0,...])
    #pprint('LSTM Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)
    
    #psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], gan_prediction[0,...])
    #pprint('GAN Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)
    
    #psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], combiner_out[0,...])
    #pprint('Combiner Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)

    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
batch_size = 30
batch_s2, batch_s1 = next(iterator)

batch_s2, batch_s1 = next(iterator)
combiner_input = np.zeros((1, 256, 256, 6))

for i in range(batch_size):    
    lstm_prediction = lstm.predict(batch_s2[i, 0:3, ...].reshape((1, 3, 256, 256, 3)))
    
    s1 = despeckling.predict(batch_s1[i, ...].reshape(4,256,256,1))

    gan_prediction = gan.predict(s1)
    gan_prediction = (np.array(gan_prediction) + 1) / 2.0

    combiner_input[0,...,:3] = lstm_prediction[0,0,...]
    combiner_input[0,...,3:6] = gan_prediction[3,...]
    
    combiner_out = combined.predict(combiner_input)
    

    fig, axes = plt.subplots(nrows = 2, ncols = 6, figsize = (30, 10))
    
    axes[0,0].imshow(batch_s2[i, 0, ...])
    axes[0,0].set_title('S2 - 1', fontsize = 24)
    axes[0,0].axis(False)
    
    axes[0,1].imshow(batch_s2[i, 1, ...])
    axes[0,1].set_title('S2 - 2', fontsize = 24)
    axes[0,1].axis(False)
    
    axes[0,2].imshow(batch_s2[i, 2, ...])
    axes[0,2].set_title('S2 - 3', fontsize = 24)
    axes[0,2].axis(False)
    
    axes[0,3].imshow(batch_s2[i, 3, ...])
    axes[0,3].set_title('S2 - 4 (GT)', fontsize = 24)
    axes[0,3].axis(False)
    
    axes[0,4].imshow(lstm_prediction[0,0,...])
    axes[0,4].set_title('LSTM', fontsize = 24)
    axes[0,4].axis(False)
    
    axes[0,5].imshow(combiner_out[0,...])
    axes[0,5].set_title('Combined', fontsize = 24)
    axes[0,5].axis(False)

    #----------------------------------------
    axes[1,0].imshow((batch_s1[i, 0, ...]+ 1) / 2.0, cmap = 'gray')
    axes[1,0].set_title('S1 - 1', fontsize = 24)
    axes[1,0].axis(False)
    
    axes[1,1].imshow((batch_s1[i, 1, ...]+ 1) / 2.0, cmap = 'gray')
    axes[1,1].set_title('S1 - 2', fontsize = 24)
    axes[1,1].axis(False)
    
    axes[1,2].imshow((batch_s1[i, 2,...]+ 1) / 2.0, cmap = 'gray')
    axes[1,2].set_title('S1 - 3', fontsize = 24)
    axes[1,2].axis(False)
    
    axes[1,3].imshow((batch_s1[i, 3,...]+ 1) / 2.0, cmap = 'gray')
    axes[1,3].set_title('S1 - 4', fontsize = 24)
    axes[1,3].axis(False)
    
    axes[1,4].imshow(gan_prediction[3,...])
    axes[1,4].set_title('GAN', fontsize = 24)
    axes[1,4].axis(False)
    
    axes[1,5].axis(False)

    plt.show()
    plt.close()
    
    
    psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], lstm_prediction[0,0,...])
    pprint('LSTM Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)
    
    psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], gan_prediction[0,...])
    pprint('GAN Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)
    
    psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], combiner_out[0,...])
    pprint('Combiner Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)

    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
batch_s2, batch_s1 = next(iterator)
combiner_input = np.zeros((1, 256, 256, 6))

for i in range(batch_size):    
    lstm_prediction = lstm.predict(batch_s2[i, 0:3, ...].reshape((1, 3, 256, 256, 3)))
    
    s1 = despeckling.predict(batch_s1[i, ...].reshape(4,256,256,1))

    gan_prediction = gan.predict(s1)
    gan_prediction = (np.array(gan_prediction) + 1) / 2.0

    combiner_input[0,...,:3] = lstm_prediction[0,0,...]
    combiner_input[0,...,3:6] = gan_prediction[3,...]
    
    combiner_out = combined.predict(combiner_input)
    

    fig, axes = plt.subplots(nrows = 2, ncols = 6, figsize = (30, 10))
    
    axes[0,0].imshow(batch_s2[i, 0, ...])
    axes[0,0].set_title('S2 - 1', fontsize = 24)
    axes[0,0].axis(False)
    
    axes[0,1].imshow(batch_s2[i, 1, ...])
    axes[0,1].set_title('S2 - 2', fontsize = 24)
    axes[0,1].axis(False)
    
    axes[0,2].imshow(batch_s2[i, 2, ...])
    axes[0,2].set_title('S2 - 3', fontsize = 24)
    axes[0,2].axis(False)
    
    axes[0,3].imshow(batch_s2[i, 3, ...])
    axes[0,3].set_title('S2 - 4 (GT)', fontsize = 24)
    axes[0,3].axis(False)
    
    axes[0,4].imshow(lstm_prediction[0,0,...])
    axes[0,4].set_title('LSTM', fontsize = 24)
    axes[0,4].axis(False)
    
    axes[0,5].imshow(combiner_out[0,...])
    axes[0,5].set_title('Combined', fontsize = 24)
    axes[0,5].axis(False)

    #----------------------------------------
    axes[1,0].imshow((batch_s1[i, 0, ...]+ 1) / 2.0, cmap = 'gray')
    axes[1,0].set_title('S1 - 1', fontsize = 24)
    axes[1,0].axis(False)
    
    axes[1,1].imshow((batch_s1[i, 1, ...]+ 1) / 2.0, cmap = 'gray')
    axes[1,1].set_title('S1 - 2', fontsize = 24)
    axes[1,1].axis(False)
    
    axes[1,2].imshow((batch_s1[i, 2,...]+ 1) / 2.0, cmap = 'gray')
    axes[1,2].set_title('S1 - 3', fontsize = 24)
    axes[1,2].axis(False)
    
    axes[1,3].imshow((batch_s1[i, 3,...]+ 1) / 2.0, cmap = 'gray')
    axes[1,3].set_title('S1 - 4', fontsize = 24)
    axes[1,3].axis(False)
    
    axes[1,4].imshow(gan_prediction[3,...])
    axes[1,4].set_title('GAN', fontsize = 24)
    axes[1,4].axis(False)
    
    axes[1,5].axis(False)

    plt.show()
    plt.close()
    
    
    psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], lstm_prediction[0,0,...])
    pprint('LSTM Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)
    
    psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], gan_prediction[0,...])
    pprint('GAN Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)
    
    psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc = testModels(batch_s2[i,3,...], combiner_out[0,...])
    pprint('Combiner Prediction', psnr, ssim, fid, csc_psnr, rmse1, sam1, uqi1, dd, cc)

    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')